In [4]:
import cv2
import os
import requests
import pandas as pd
from tqdm.notebook import trange, tqdm
from os.path import join
import shutil
from scripts.legacy import frames_preprocess

In [5]:
root_folder = "C:/Users/Simon/Data/BOON DR CAMP PROJECT 2019-2021/"
sorted_dir = join(root_folder, "02_sorted_by_eye_pp/")
renamed_dir = join(root_folder, "03_renamed_and_sorted_pp")
created_folders = os.listdir(sorted_dir)
OPT_RE = 'OPTOMETRIST \nRE :  DR GRADE - PAXOSCOPE POST-DILATION'
OPT_LE = 'OPTOMETRIST LE :  DR GRADE - PAXOSCOPE POST-DILATION'
OPH_RE = 'Ophthalmologist\nRE TELEGRADING'
OPH_LE = 'Ophthalmologist\nLE TELEGRADING'

df = pd.read_excel(join(root_folder, "BOON DR CAMP PROJECT 2019-2021_preparation for R 1.1.xlsx"))
df = df.drop(df.columns[-29:],axis=1)
print(df.columns)
print(f'Dataset length: {len(df)}')
#df.head()

Index(['DATE OF EXAMINATION', 'CAMP LOCATION', 'PATIENT ID', 'GENDER',
       'AGE (YEARS)', 'EDUCATION', 'BP', 'RBS', 'HEIGHT (cm)', 'WEIGHT (kg)',
       'HYPERTENSION', 'DIABETES', 'FAMILY HISTORY OF DIABETES',
       'DURATION OF DIABETES', 'DIABETIC MEDICATION',
       'DIABETIC MEDICATION GROUP',
       'SYSTEMIC ABNORMALITY \nOPTIONS- NIL, CARDIAC, NEPHROPATHY, STROKE, OTHERS',
       'RE VISUAL ACUITY', 'LE VISUAL ACUITY', 'RE LENS STATUS',
       'LE LENS STATUS', 'RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'RE PAXOS DISC CENTRED IMAGE UPLOAD', 'RE PAXOS TEMPORAL IMAGE UPLOAD',
       'RE PAXOS STQ IMAGE UPLOAD', 'RE PAXOS ITQ IMAGE UPLOAD',
       'RE PAXOS SNQ IMAGE UPLOAD', 'RE PAXOS INQ IMAGE UPLOAD',
       'RE PAXOS OTHER IMAGES UPLOAD',
       'LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'LE PAXOS DISC CENTRED IMAGE UPLOAD', 'LE PAXOS TEMPORAL IMAGE UPLOAD',
       'LE PAXOS STQ IMAGE UPLOAD', 'LE PAXOS ITQ IMAGE UPLOAD',
       'LE PAXOS SNQ IMAGE UPLOAD',

In [6]:
# Remove empty rows
'''df_data = df.loc[1:, ['PATIENT ID','RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'RE PAXOS DISC CENTRED IMAGE UPLOAD', 'RE PAXOS TEMPORAL IMAGE UPLOAD',
       'RE PAXOS STQ IMAGE UPLOAD', 'RE PAXOS ITQ IMAGE UPLOAD',
       'RE PAXOS SNQ IMAGE UPLOAD', 'RE PAXOS INQ IMAGE UPLOAD',
       'RE PAXOS OTHER IMAGES UPLOAD',
       'LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD',
       'LE PAXOS DISC CENTRED IMAGE UPLOAD', 'LE PAXOS TEMPORAL IMAGE UPLOAD',
       'LE PAXOS STQ IMAGE UPLOAD', 'LE PAXOS ITQ IMAGE UPLOAD',
       'LE PAXOS SNQ IMAGE UPLOAD', 'LE PAXOS INQ IMAGE UPLOAD',
       'LE PAXOS OTHER IMAGES UPLOAD']]'''

df_data = df.loc[1:, :]
df_data = df_data.drop(df_data[df_data['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'] == '[]'].index)
df_data = df_data.drop(df_data[df_data['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'] == 'null'].index)
df_data = df_data[(df_data['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'].notna() | df_data['LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'].notna())]
#df_data.tail()
print(f'Cleaned dataset length: {len(df_data)}')
df_data.to_excel(join(root_folder, "BOON DR CAMP PROJECT 2019-2021 cleaned 1.1.xlsx"), index=False)

Cleaned dataset length: 223


In [41]:
# attempt to download data from aws
base_url = 'https://teleopticsapp.s3.ap-south-1.amazonaws.com/'
base_folder = 'aws'
failed_attempts = 0

if not os.path.exists(base_folder):
    os.mkdir(base_folder)

for i, row in tqdm(df_data.iterrows(), total=len(df_data)):
    if row['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'][0] != '[':
        # skip locally saved files
        continue
    url_list = eval(row['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD'])
    for url in url_list:
        res = requests.get(url)
        img_data = res.content
        file_name = url.replace(base_url, '')
        file_name = file_name.replace('/', '_')
        if not res.ok:
            print(f'Failed download from {url}')
            failed_attempts += 1
        else: 
            with open(os.path.join(base_folder, file_name), 'wb') as h:
                h.write(img_data)
        
    

  0%|          | 0/219 [00:00<?, ?it/s]

Failed download from https://teleopticsapp.s3.ap-south-1.amazonaws.com/CZ20211022003/RE/POST/REPOSTMC_20211024_237.jpg


In [4]:
# Check whether tele-opthmologists and optometrist grading is the same
df_data = df_data.replace('CANT GRADE ', '')
df_data = df_data.replace('CANNOT COMMENT', '')

for i, row in tqdm(df_data.iterrows(), total=len(df_data)):
    if pd.isna(row[OPT_RE]) or pd.isna(row[OPH_RE]):
        pass
    elif not row[OPT_RE].split(' ')[0].lower() == row[OPH_RE].split(' ')[0].lower():
        print(f"Conflicting grading for {i}, {row['PATIENT ID']}, right eye: {row[OPT_RE]}/{row[OPH_RE]}")
    if pd.isna(row[OPT_LE]) or pd.isna(row[OPH_LE]):
        pass
    elif not row[OPT_LE].split(' ')[0].lower() == row[OPH_LE].split(' ')[0].lower():
        print(f"Conflicting grading for {i}, {row['PATIENT ID']}, left eye: {row[OPT_LE]}/{row[OPH_LE]}")

# 81, 160421003 -> NO DR
# 71, 1311200002 -> NO DR
# 62, 280220003RR -> MODERATE NPDR
# 60, 280220001PR -> MILD NPDR
# 56, 220220001PR -> MODERATE NPDR
# 46, 020220001KA -> MODERATE NPDR
# 45, 02022002PA -> MILD NPDR
# 44, 120120008SK -> MILD NPDR
# 34, DR270120001A -> MILD NPDR
# 27, 191220002BSR -> NO DR
# 26, 191220001RM -> MILD NPDR
# 23, 191215011 -> CANNOT COMMENT
# 19, 191215006GJ -> MILD NPDR
# 18, 191215005VS -> NO DR/MILD NPDR
# 17, 191215004Z -> RE: CANNOT COMMENT, LE: MODERATE NPDR

  0%|          | 0/223 [00:00<?, ?it/s]

Conflicting grading for 17, 191215004Z, right eye: /MODERATE NPDR
Conflicting grading for 17, 191215004Z, left eye: SEVERE NPDR/MODERATE NPDR
Conflicting grading for 18, 191215005VS, right eye: NO DR/MILD NPDR
Conflicting grading for 19, 191215006GJ, right eye: MODERATE NPDR/MILD NPDR
Conflicting grading for 20, 191215008AA, right eye: NO DR/
Conflicting grading for 23, 191215011, left eye: /NO DR
Conflicting grading for 26, 191220001RM, left eye: Moderate NPDR/MILD NPDR
Conflicting grading for 27, 191220002BSR, right eye: NO DR/
Conflicting grading for 34, DR270120001A, right eye: mild NPDR/NO DR
Conflicting grading for 44, 120120008SK, right eye: NO DR/MILD NPDR
Conflicting grading for 45, 02022002PA, left eye: NO DR/MILD NPDR
Conflicting grading for 46, 020220001KA, right eye: MILD NPDR/MODERATE NPDR
Conflicting grading for 56, 220220001PR, right eye: SEVERE NPDR/MODERATE NPDR
Conflicting grading for 56, 220220001PR, left eye: SEVERE NPDR/MODERATE NPDR
Conflicting grading for 60, 28

In [5]:
# Move images to examination folder according to excel file
dir_name = join(root_folder, "02_sorted_by_eye_pp/")
lookup_folder = '01_images_pp' # for non-processed images'SANKARA UKB DR STUDY_Images'
right_eye_cols = ['RE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD','RE PAXOS DISC CENTRED IMAGE UPLOAD', 'RE PAXOS TEMPORAL IMAGE UPLOAD',
       'RE PAXOS STQ IMAGE UPLOAD', 'RE PAXOS ITQ IMAGE UPLOAD', 'RE PAXOS SNQ IMAGE UPLOAD', 'RE PAXOS INQ IMAGE UPLOAD',
       'RE PAXOS OTHER IMAGES UPLOAD']
left_eye_cols = ['LE PAXOSCOPE MACULA CENTRED IMAGE UPLOAD', 'LE PAXOS DISC CENTRED IMAGE UPLOAD', 'LE PAXOS TEMPORAL IMAGE UPLOAD',
       'LE PAXOS STQ IMAGE UPLOAD', 'LE PAXOS ITQ IMAGE UPLOAD', 'LE PAXOS SNQ IMAGE UPLOAD', 'LE PAXOS INQ IMAGE UPLOAD',
       'LE PAXOS OTHER IMAGES UPLOAD']

if os.path.exists(dir_name):
    shutil.rmtree(dir_name)
os.mkdir(dir_name)
for i, row in tqdm(df_data.iterrows(), total=len(df_data)):
    pid = row['PATIENT ID']
    if i > 82:
        continue
    dir_le, dir_re = join(dir_name, str(pid) + '_LE'), join(dir_name, str(pid) + '_RE')
    os.mkdir(dir_le)
    os.mkdir(dir_re)
    
    for j, col in enumerate(right_eye_cols):
        if not pd.isna(row[col]): 
            row[col] = join(lookup_folder, row[col].split('/')[1])
        if not pd.isna(row[col]) and os.path.exists(join(root_folder, row[col])):
            shutil.copy(join(root_folder, row[col]), join(dir_re, os.path.basename(row[col])))
    for j, col in enumerate(left_eye_cols):
        if not pd.isna(row[col]):
            row[col] = join(lookup_folder, row[col].split('/')[1])
        if not pd.isna(row[col]) and os.path.exists(join(root_folder, row[col])):
            shutil.copy(join(root_folder, row[col]), join(dir_le, os.path.basename(row[col])))

created_folders = os.listdir(dir_name)
deletions = 0
for folder in created_folders:
    if len(os.listdir(join(dir_name, folder))) == 0:
        os.rmdir(join(dir_name, folder))
        deletions += 1
print(f'{deletions} emtpy folders were removed after population')

  0%|          | 0/223 [00:00<?, ?it/s]

6 emtpy folders were removed after population


In [40]:
# translate filenames to Paxos nomentclature: ([A-Z])(\d){3}[RL](\d)?
camp_prefix = 'X'
os.mkdir(renamed_dir)
count = 0
seen = {}
for i, folder in enumerate(created_folders):
    old_count, count_flag = -1, False
    if folder.split('_')[0] not in seen.keys():
        count += 1
        seen[folder.split('_')[0]] = count
    else:
        count = seen[folder.split('_')[0]]
        old_count = count
        count_flag = True

    suffix = folder.split('_')[1][0]
    exam_id = f'{camp_prefix}{count:03d}{suffix}'
    os.mkdir(join(renamed_dir, exam_id))

    for j, img in enumerate(os.listdir(join(sorted_dir, folder))):
        shutil.copy(join(sorted_dir, folder, img), join(renamed_dir, exam_id, f'{exam_id}_{j:02d}{os.path.splitext(img)[1]}'))
        # Copy file just into the root folder
        shutil.copy(join(sorted_dir, folder, img), join(renamed_dir, f'{exam_id}_{j:02d}{os.path.splitext(img)[1]}'))
    with open(join(renamed_dir, exam_id, 'legacy.txt'), "w") as f:
        f.write(folder)
    if count_flag:
        count = old_count
        count_flag = False

In [1]:
# 1200 files
# 1171 lose 30 files through cropping
# 823 lose 340 files which are not mentioned in the excel file

In [41]:
# Create label file (Map new_id to old_id labeling info, copy relevant labels)
df_labels = pd.DataFrame(columns=['image', 'level', 'optometrist_rating', 'ophthalmologist_rating'])
df_labels_exams = pd.DataFrame(columns=['image', 'level', 'optometrist_rating', 'ophthalmologist_rating'])
for i, folder in tqdm(enumerate(os.listdir(renamed_dir)), total=len(os.listdir(renamed_dir))):
    if not os.path.exists(join(renamed_dir, folder, 'legacy.txt')):
        continue
    with open(join(renamed_dir, folder, 'legacy.txt'), 'r') as f:
        old_id = f.read()
        if len(old_id) > 0:
            old_id = old_id.split('_')[0]

    corres_row = df_data.loc[df_data['PATIENT ID'] == old_id].to_dict()
    #print(old_id, list(corres_row[OPT_LE].values())[0],  corres_row[OPT_RE],  corres_row[OPH_LE],  corres_row[OPH_RE])
    face_side = folder[-1]
    new_entry = {'optometrist_rating': None, 'ophthalmologist_rating': None, 'level': None, 'image': None}
    for img_path in os.listdir(join(renamed_dir, folder)):
        if img_path == 'legacy.txt':
            continue
        if face_side == 'L':
            opt = OPT_LE
            oph = OPH_LE
        elif face_side == 'R':
            opt = OPT_RE
            oph = OPH_RE
        else:
            oph, opt = None, None
        # print(old_id, face_side, corres_row[opt].values())
        if len(list(corres_row[opt].values())) > 0:
            new_entry['optometrist_rating'] = list(corres_row[opt].values())[0]
            if type(new_entry['optometrist_rating']) == str:
                new_entry['optometrist_rating'] = new_entry['optometrist_rating'].lower()
        if len(list(corres_row[opt].values())) > 0:
            new_entry['ophthalmologist_rating'] = list(corres_row[oph].values())[0]
            if type(new_entry['ophthalmologist_rating']) == str:
                new_entry['ophthalmologist_rating'] = new_entry['ophthalmologist_rating'].lower()
        #if new_entry['ophthalmologist_rating'] == new_entry['optometrist_rating']:
        if new_entry['ophthalmologist_rating']:
            new_entry['level'] = new_entry['ophthalmologist_rating']
        elif new_entry['optometrist_rating']:
            new_entry['level'] = new_entry['optometrist_rating']
        new_entry['image'] = img_path

        df_labels = df_labels.append(new_entry, ignore_index=True)
    df_labels_exams = df_labels_exams.append({'image': folder, 'level': new_entry['level']}, ignore_index=True)

df_labels.to_csv(join(root_folder, renamed_dir, 'labels_renamed.csv'), index=False)
df_labels_exams.to_csv(join(root_folder, renamed_dir, 'labels_exam_renamed.csv'), index=False)

  0%|          | 0/957 [00:00<?, ?it/s]

In [13]:
# Process images from AWS cloud
additional_labels = join(root_folder, 'UPDATED DR GRADING BONN PROJECT.xlsx')
raw_data = join(root_folder, 'SANKARA UKB DR STUDY_aws/')
preprocessed_data = join(root_folder, '01_images_aws/')
cropped_data = join(root_folder, '02_images_aws_pp/')
sorted_data = join(root_folder, '03_images_aws_pp_sorted/')
renamed_data = join(root_folder, '04_images_aws_pp_renamed/')
first_index =  71

if not os.path.exists(preprocessed_data):
    os.mkdir(preprocessed_data)
    for f in tqdm(os.listdir(raw_data), total=len(os.listdir(raw_data))):
        img = cv2.imread(join(raw_data, f))
        if img.shape[0] < 3000:
            shutil.copy(join(raw_data, f), join(preprocessed_data, f))
            continue
        img = cv2.resize(img, dsize=None, fx=0.4, fy=0.4)
        cv2.imwrite(join(preprocessed_data, f), img)

if not os.path.exists(cropped_data):
    os.mkdir(cropped_data)
    frames_preprocess.run(preprocessed_data, cropped_data, min_radius=350)

folders = []
if not os.path.exists(sorted_data):
    os.mkdir(sorted_data)
    for i, f in enumerate(os.listdir(cropped_data)):
        img_id = f.split('_')[0]
        suffix = f.split('_')[1][0]
        exam_id = f'{img_id}_{suffix}'

        if exam_id in folders:
            shutil.copy(join(cropped_data, f), join(sorted_data, exam_id, f))
        else:
            os.mkdir(join(sorted_data, exam_id))
            shutil.copy(join(cropped_data, f), join(sorted_data, exam_id, f))
            folders.append(exam_id)

cur_index = first_index
if not os.path.exists(renamed_data):
    os.mkdir(renamed_data)
    for i, folder in enumerate(os.listdir(sorted_data)):
        suffix = folder.split('_')[1]
        exam_id = f'{camp_prefix}{cur_index:03d}{suffix}'
        os.mkdir(join(renamed_data, exam_id))

        for j, img in enumerate(os.listdir(join(sorted_data, folder))):
            shutil.copy(join(sorted_data, folder, img), join(renamed_data, exam_id, f'{exam_id}_{j:02d}{os.path.splitext(img)[1]}'))
            # Copy file just into the root folder
            shutil.copy(join(sorted_data, folder, img), join(renamed_data, f'{exam_id}_{j:02d}{os.path.splitext(img)[1]}'))
        with open(join(renamed_data, exam_id, 'legacy.txt'), "w") as f:
            f.write(folder)
        cur_index += 1

    #create lables file
    ref_df = pd.read_excel(additional_labels)
    aws_df = pd.DataFrame(columns=['image', 'level', 'optometrist_rating', 'ophthalmologist_rating'])
    for i, folder in enumerate(os.listdir(renamed_data)):
        if not os.path.exists(join(renamed_data, folder, 'legacy.txt')):
            continue
        with open(join(renamed_data, folder, 'legacy.txt'), 'r') as f:
            old_id = f.read()
            if len(old_id) > 0:
                old_id = old_id.split('_')[0]
        corres_row = ref_df.loc[ref_df['PATIENT ID'] == old_id].to_dict()
        face_side = folder[-1]
        aws_df = aws_df.append({
            'image': folder,
            'level': list(corres_row['LE TELEGRADING' if face_side == 'L' else 'RE TELEGRADING'].values())[0]
        }, ignore_index=True)
    aws_df.to_csv(join(root_folder, renamed_data, 'labels_aws_renamed.csv'), index=False)


# RE TELEGRADING	LE TELEGRADING

In [24]:
# Combined aws and hard-drive images in one dataset
combined_dir = join(root_folder, '05_images_pp_renamed_combined')
# os.mkdir(combined_dir)
if not os.path.exists(combined_dir):
    shutil.copytree(renamed_dir, combined_dir, copy_function = shutil.copy)
    shutil.copytree(renamed_data, combined_dir, copy_function = shutil.copy, dirs_exist_ok=True)

df_aws = pd.read_csv(join(root_folder, combined_dir, 'labels_aws_renamed.csv'))
df_hdd = pd.read_csv(join(root_folder, combined_dir, 'labels_exam_renamed.csv'))
df_combined = pd.concat([df_hdd, df_aws], ignore_index=True)
df_combined = df_combined.drop(df_combined[(df_combined.level == 'POOR QUALITY') | (df_combined.level == 'NO MEDIA')].index)
df_combined.to_csv(join(root_folder, combined_dir, 'labels_combined.csv'), index=False)